In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect("../data/aml_simulation.db")
export_dir = "../data/output/pbi_exports"

In [3]:
query = """
WITH joined_data AS (
    SELECT
        f.flagged_id,
        f.rule_triggered,
        f.reason,
        
        t.txn_id,
        t.timestamp,
        t.amount,
        t.currency,
        t.origin_country,
        t.dest_country,
        t.channel,
        t.counterparty,

        a.account_id,
        a.account_type,
        a.open_date,

        c.customer_id,
        c.name AS customer_name,
        c.country AS customer_country,
        c.risk_category,
        c.dob
    FROM flagged_txns f
    JOIN transactions t ON f.txn_id = t.txn_id
    JOIN accounts a ON t.account_id = a.account_id
    JOIN customers c ON a.customer_id = c.customer_id
    WHERE DATE(t.timestamp) >= DATE('now', '-180 days')
),

ranked_data AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY txn_id ORDER BY timestamp DESC) AS row_num,
           COUNT(*) OVER (PARTITION BY customer_id) AS txn_count_per_customer,
           DENSE_RANK() OVER (PARTITION BY rule_triggered ORDER BY timestamp DESC) AS latest_hit_order
    FROM joined_data
)

SELECT *
FROM ranked_data
WHERE row_num = 1
ORDER BY timestamp DESC;
"""

In [4]:
df = pd.read_sql_query(query, conn)
df.to_csv(f"{export_dir}/flagged_risk_events_cte.csv", index=False)
conn.close()